In [ ]:
# GPT API & LangChain 연동 멀티모달 AI 시스템 (API 키 직접 설정)
# 이미지 업로드, 분석, 질문답변을 모두 웹에서 할 수 있는 통합 시스템

# 필요한 라이브러리 설치
import subprocess
import sys

def install_packages():
    """필요한 패키지들을 설치"""
    packages = [
        "gradio",
        "torch",
        "torchvision",
        "transformers",
        "pillow",
        "requests",
        "matplotlib",
        "openai",
        "langchain",
        "langchain-openai",
        "langsmith",
        "langchain-community",
        "tavily-python"
    ]

    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        except subprocess.CalledProcessError:
            print(f"Warning: Failed to install {package}")

# langchain-teddynote 설치 (TavilySearch용)
def install_teddynote():
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain-teddynote"])
    except subprocess.CalledProcessError:
        print("Warning: Failed to install langchain-teddynote")

# 패키지 설치 실행
install_packages()
install_teddynote()

# CLIP 설치 (별도로)
try:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "git+https://github.com/openai/CLIP.git"])
except:
    print("Warning: CLIP installation failed, some features may be limited")

import os
import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50, ResNet50_Weights
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
import requests
import io
import numpy as np
import warnings

# =============================================================================
# API 키 설정 (사용자가 직접 입력)
# =============================================================================

# OpenAI API 키 설정 (사용자가 입력해야 함)
# from google.colab import userdata

# api_key=userdata.get('api_key')
# api_key2=userdata.get('api_key2')
# api_key3=userdata.get('api_key3')
# os.environ["OPENAI_API_KEY"] = api_key
# os.environ["LANGCHAIN_API_KEY"] = api_key2
# os.environ["TAVILY_API_KEY"] = api_key3

from dotenv import load_dotenv

load_dotenv()
# OpenAI API 클라이언트 생성
OPENAPI_KEY = os.getenv("OPENAI_API_KEY")
LangSmith_KEY = os.getenv("LANGCHAIN_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

# 2) LangSmith 연동 설정
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "langgraph_6"

# =============================================================================
# 라이브러리 Import 및 모델 가용성 확인
# =============================================================================

# OpenAI & LangChain imports
from openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory
from langchain.callbacks import LangChainTracer
from langsmith import Client

# Tavily Search import
try:
    from langchain_teddynote.tools.tavily import TavilySearch
    TAVILY_AVAILABLE = True
    print("✅ Tavily Search (langchain-teddynote) 로드 완료")
except ImportError:
    try:
        from langchain_community.tools.tavily_search import TavilySearchResults
        TAVILY_AVAILABLE = True
        print("✅ Tavily Search (community) 로드 완료")
    except ImportError:
        TAVILY_AVAILABLE = False
        print("❌ Tavily Search not available")

# CLIP import
try:
    import clip
    CLIP_AVAILABLE = True
    print("✅ CLIP 라이브러리 로드 완료")
except ImportError:
    CLIP_AVAILABLE = False
    print("❌ CLIP not available, using alternative methods")

warnings.filterwarnings('ignore')

class GPTMultimodalAI:
    """GPT API 연동 멀티모달 AI 시스템 (API 키 직접 설정)"""

    def __init__(self):
        print("🚀 GPT API 연동 AI 시스템 초기화 중...")

        # 모델 가용성 상태 초기화
        self.clip_available = False
        self.blip_available = False
        self.search_available = False

        # API 키 확인 및 설정
        self.setup_api_services()

        # 디바이스 설정
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"🖥️ 사용 디바이스: {self.device}")

        # 모델들 초기화
        self.init_classification_model()
        self.init_vision_language_models()
        self.init_gpt_and_langchain()
        self.init_search_tools()

        self.current_image = None
        self.analysis_cache = {}

        print("✅ AI 시스템 초기화 완료!")

    def setup_api_services(self):
        """API 서비스 설정"""
        print("🔑 API 서비스 설정 중...")

        # OpenAI 클라이언트 초기화
        self.openai_api_key = OPENAI_API_KEY
        self.openai_client = OpenAI(api_key=self.openai_api_key)

        # LangSmith 설정 확인
        self.langsmith_api_key = os.environ.get("LANGCHAIN_API_KEY")
        self.langsmith_project = os.environ.get("LANGSMITH_PROJECT", "langgraph_5")

        print(f"✅ OpenAI API: 설정됨")
        print(f"✅ LangSmith: 설정됨 (프로젝트: {self.langsmith_project})")
        print(f"✅ Tavily API: 설정됨")

    def init_classification_model(self):
        """이미지 분류 모델 초기화"""
        print("📦 이미지 분류 모델 로딩...")
        try:
            self.classification_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
            self.classification_model.eval()

            self.classification_transform = transforms.Compose([
                transforms.Resize(256),
                transforms.CenterCrop(224),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])

            # ImageNet 클래스 로드
            try:
                url = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
                response = requests.get(url, timeout=10)
                self.imagenet_classes = response.text.strip().split('\n')
            except:
                self.imagenet_classes = [f"클래스_{i}" for i in range(1000)]

            print("✅ ResNet 분류 모델 로드 완료")
        except Exception as e:
            print(f"❌ 분류 모델 로드 실패: {e}")

    def init_vision_language_models(self):
        """비전-언어 모델 초기화"""
        print("🔍 비전-언어 모델 로딩...")

        # CLIP 모델 초기화
        if CLIP_AVAILABLE:
            try:
                self.clip_model, self.clip_preprocess = clip.load("ViT-B/32", device=self.device)
                self.clip_available = True
                print("✅ CLIP 모델 로드 완료")
            except Exception as e:
                self.clip_available = False
                print(f"❌ CLIP 모델 로드 실패: {e}")
        else:
            self.clip_available = False
            print("ℹ️ CLIP 라이브러리를 사용할 수 없습니다")

        # BLIP 모델 초기화
        try:
            self.blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
            self.blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
            self.blip_available = True
            print("✅ BLIP 모델 로드 완료")
        except Exception as e:
            self.blip_available = False
            print(f"❌ BLIP 모델 로드 실패: {e}")

    def init_gpt_and_langchain(self):
        """GPT 및 LangChain 초기화"""
        print("🧠 GPT 및 LangChain 초기화...")

        try:
            # LangChain ChatOpenAI 설정
            self.llm = ChatOpenAI(
                openai_api_key=self.openai_api_key,
                model_name="gpt-3.5-turbo",
                temperature=0.7,
                max_tokens=1000
            )

            # 메모리 설정
            self.memory = ConversationBufferMemory(
                return_messages=True,
                memory_key="chat_history"
            )

            # LangSmith 트레이서 설정
            try:
                self.tracer = LangChainTracer(
                    project_name=self.langsmith_project
                )
                self.callbacks = [self.tracer]
                print(f"✅ LangSmith 트레이싱 활성화 (프로젝트: {self.langsmith_project})")
            except:
                self.callbacks = []
                print("⚠️ LangSmith 트레이싱 설정 실패")

            print("✅ GPT 및 LangChain 초기화 완료")

        except Exception as e:
            print(f"❌ GPT/LangChain 초기화 실패: {e}")
            self.llm = None

    def init_search_tools(self):
        """검색 도구 초기화"""
        print("🔍 검색 도구 초기화...")

        if TAVILY_AVAILABLE:
            try:
                # TavilySearch 초기화 시도
                if 'TavilySearch' in globals():
                    self.tavily_search = TavilySearch()
                else:
                    # 대안으로 TavilySearchResults 사용
                    self.tavily_search = TavilySearchResults(max_results=3)
                self.search_available = True
                print("✅ Tavily Search 도구 로드 완료")
            except Exception as e:
                self.search_available = False
                print(f"⚠️ Tavily Search 초기화 실패: {e}")
        else:
            self.search_available = False
            print("❌ Tavily Search 사용 불가")

    def search_web(self, query):
        """웹 검색 수행"""
        if not self.search_available:
            return "웹 검색 기능이 사용 불가능합니다."

        try:
            # TavilySearch 실행
            if hasattr(self, 'tavily_search'):
                if hasattr(self.tavily_search, 'invoke'):
                    results = self.tavily_search.invoke(query)
                else:
                    results = self.tavily_search.run(query)

                # 결과 정리
                if isinstance(results, list):
                    search_summary = "\n".join([
                        f"• {result.get('title', 'No title')}: {result.get('content', result.get('snippet', 'No content'))[:200]}..."
                        for result in results[:3]
                    ])
                elif isinstance(results, str):
                    search_summary = results[:500] + "..." if len(results) > 500 else results
                else:
                    search_summary = str(results)[:500]

                return search_summary
            else:
                return "검색 도구가 초기화되지 않았습니다."

        except Exception as e:
            return f"검색 중 오류 발생: {str(e)}"

    def classify_image(self, image):
        """이미지 분류 수행"""
        if image is None:
            return "이미지를 업로드해주세요."

        try:
            # PIL Image로 변환
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif not isinstance(image, Image.Image):
                image = Image.fromarray(image).convert('RGB')

            # 전처리 및 예측
            input_tensor = self.classification_transform(image).unsqueeze(0)

            with torch.no_grad():
                outputs = self.classification_model(input_tensor)
                probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

            # 상위 5개 결과
            top_probs, top_indices = torch.topk(probabilities, 5)

            results = "🎯 **이미지 분류 결과**\n\n"
            for i in range(5):
                class_idx = top_indices[i].item()
                prob = top_probs[i].item()
                class_name = self.imagenet_classes[class_idx]
                results += f"**{i+1}.** {class_name}\n"
                results += f"   📊 신뢰도: {prob*100:.2f}%\n\n"

            self.current_image = image
            return results

        except Exception as e:
            return f"❌ 분류 중 오류 발생: {str(e)}"

    def generate_description_with_gpt(self, image):
        """GPT API를 사용한 이미지 설명 생성"""
        if image is None:
            return "이미지를 업로드해주세요."

        try:
            # PIL Image로 변환
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif not isinstance(image, Image.Image):
                image = Image.fromarray(image).convert('RGB')

            # BLIP으로 기본 캡션 생성
            basic_caption = self.get_basic_caption(image)

            # 이미지 분석 정보 수집
            analysis_info = self.get_image_analysis(image)

            # GPT로 향상된 설명 생성
            enhanced_description = self.enhance_description_with_gpt(basic_caption, analysis_info)

            result = "📖 **GPT 향상 이미지 설명**\n\n"
            result += f"🔍 **기본 분석:** {basic_caption}\n\n"
            result += f"🧠 **GPT 향상 설명:**\n{enhanced_description}\n\n"
            result += f"📊 **분석 정보:** {analysis_info}"

            self.current_image = image
            return result

        except Exception as e:
            return f"❌ 설명 생성 중 오류 발생: {str(e)}"

    def get_basic_caption(self, image):
        """BLIP을 사용한 기본 캡션 생성"""
        if not self.blip_available:
            return "이미지 분석 중 (BLIP 모델 사용 불가)"

        try:
            inputs = self.blip_processor(image, return_tensors="pt")
            with torch.no_grad():
                out = self.blip_model.generate(**inputs, max_length=50)
            return self.blip_processor.decode(out[0], skip_special_tokens=True)
        except Exception as e:
            return f"이미지 분석 중 (오류: {str(e)[:50]})"

    def get_image_analysis(self, image):
        """이미지 추가 분석 정보"""
        analysis = []

        # 이미지 크기
        width, height = image.size
        analysis.append(f"크기: {width}x{height}")

        # 색상 분석
        try:
            colors = image.getcolors(maxcolors=256*256*256)
            if colors:
                analysis.append(f"색상 분석 완료")
        except:
            analysis.append("색상 분석 실패")

        # CLIP 분석 (가능한 경우)
        if self.clip_available and hasattr(self, 'clip_model'):
            try:
                categories = ["사람", "동물", "자연", "건물", "음식", "차량", "스포츠", "예술"]
                category = self.analyze_with_clip(image, categories)
                analysis.append(f"카테고리: {category}")
            except:
                analysis.append("CLIP 분석 실패")

        return " | ".join(analysis)

    def analyze_with_clip(self, image, categories):
        """CLIP을 사용한 카테고리 분석"""
        if not self.clip_available:
            return "CLIP 사용 불가"

        try:
            text_queries = [f"a photo of {cat}" for cat in categories]
            text_tokens = clip.tokenize(text_queries).to(self.device)
            image_tensor = self.clip_preprocess(image).unsqueeze(0).to(self.device)

            with torch.no_grad():
                image_features = self.clip_model.encode_image(image_tensor)
                text_features = self.clip_model.encode_text(text_tokens)
                similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)

            best_idx = similarity[0].argmax().item()
            return categories[best_idx]
        except Exception as e:
            return f"분석 오류: {str(e)[:20]}"

    def enhance_description_with_gpt(self, basic_caption, analysis_info):
        """GPT로 설명 향상"""
        if not self.llm:
            return "GPT API가 설정되지 않았습니다."

        try:
            prompt = f"""
            이미지 기본 분석: {basic_caption}
            추가 정보: {analysis_info}

            위 정보를 바탕으로 이미지에 대한 상세하고 흥미로운 한국어 설명을 작성해주세요.
            다음 요소들을 포함해주세요:
            1. 시각적 요소들의 구체적 설명
            2. 분위기나 느낌
            3. 주목할만한 특징들
            4. 추정되는 상황이나 맥락

            자연스러운 한국어로 2-3 문단으로 작성해주세요.
            """

            messages = [
                SystemMessage(content="당신은 이미지를 분석하고 창의적이고 상세한 설명을 제공하는 전문가입니다."),
                HumanMessage(content=prompt)
            ]

            response = self.llm.invoke(messages, callbacks=self.callbacks)
            return response.content

        except Exception as e:
            return f"GPT 설명 생성 중 오류: {str(e)}"

    def answer_question_with_gpt_and_search(self, image, question):
        """GPT API와 웹 검색을 결합한 이미지 질문답변"""
        if image is None:
            return "먼저 이미지를 업로드해주세요."

        if not question or question.strip() == "":
            return "질문을 입력해주세요."

        if not self.llm:
            return "GPT API가 설정되지 않았습니다."

        try:
            # PIL Image로 변환
            if isinstance(image, str):
                image = Image.open(image).convert('RGB')
            elif not isinstance(image, Image.Image):
                image = Image.fromarray(image).convert('RGB')

            # 이미지 컨텍스트 생성
            context = self.create_image_context_for_gpt(image)

            # 필요시 웹 검색 수행
            search_results = ""
            if any(keyword in question.lower() for keyword in ['최신', '현재', '뉴스', '정보', '찾아', '검색']):
                search_query = f"{context['caption']} {question}"
                search_results = self.search_web(search_query)

            # GPT로 질문답변
            answer = self.generate_gpt_answer_with_search(context, question, search_results)

            result = f"❓ **질문:** {question}\n\n"
            result += f"🤖 **GPT 답변:** {answer}"

            if search_results and search_results != "웹 검색 기능이 사용 불가능합니다.":
                result += f"\n\n🔍 **참고 검색 결과:**\n{search_results[:300]}..."

            self.current_image = image
            return result

        except Exception as e:
            return f"❌ 답변 생성 중 오류 발생: {str(e)}"

    def create_image_context_for_gpt(self, image):
        """GPT용 이미지 컨텍스트 생성"""
        context = {}

        # 기본 캡션
        context['caption'] = self.get_basic_caption(image)

        # 분류 정보
        try:
            input_tensor = self.classification_transform(image).unsqueeze(0)
            with torch.no_grad():
                outputs = self.classification_model(input_tensor)
                probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
            top_prob, top_idx = torch.topk(probabilities, 3)

            top_classes = []
            for i in range(3):
                class_name = self.imagenet_classes[top_idx[i].item()]
                prob = top_prob[i].item()
                top_classes.append(f"{class_name} ({prob*100:.1f}%)")

            context['classification'] = ", ".join(top_classes)
        except:
            context['classification'] = "분류 정보 없음"

        # 추가 분석
        context['analysis'] = self.get_image_analysis(image)

        return context

    def generate_gpt_answer_with_search(self, context, question, search_results=""):
        """GPT로 검색 결과를 포함한 질문 답변 생성"""
        try:
            search_context = f"\n\n웹 검색 결과:\n{search_results}" if search_results else ""

            prompt = f"""
            이미지 분석 정보:
            - 기본 설명: {context['caption']}
            - 분류 결과: {context['classification']}
            - 추가 분석: {context['analysis']}{search_context}

            사용자 질문: {question}

            위 이미지 분석 정보와 검색 결과를 바탕으로 사용자의 질문에 정확하고 도움이 되는 한국어 답변을 제공해주세요.
            만약 이미지 정보만으로는 확실한 답변이 어려운 경우, 그렇다고 명시하고 가능한 추정이나 일반적인 정보를 제공해주세요.
            검색 결과가 있다면 이를 적절히 활용해주세요.
            """

            # 메모리에서 이전 대화 가져오기
            chat_history = self.memory.chat_memory.messages if self.memory.chat_memory.messages else []

            messages = [
                SystemMessage(content="당신은 이미지를 분석하고 웹 검색 결과를 활용하여 사용자의 질문에 정확하게 답변하는 AI 어시스턴트입니다."),
                *chat_history[-6:],  # 최근 3번의 대화만 유지
                HumanMessage(content=prompt)
            ]

            # callbacks는 이미 초기화 시 설정되어 있으므로 제거
            response = self.llm.invoke(messages)
            answer = response.content

            # 메모리에 대화 저장
            self.memory.chat_memory.add_user_message(question)
            self.memory.chat_memory.add_ai_message(answer)

            return answer

        except Exception as e:
            return f"GPT 답변 생성 중 오류: {str(e)}"

    def get_api_status(self):
        """API 상태 확인"""
        status = "🔧 **API 상태**\n\n"

        # OpenAI API 상태
        try:
            test_response = self.openai_client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": "test"}],
                max_tokens=5
            )
            status += "✅ OpenAI API: 연결됨\n"
        except:
            status += "❌ OpenAI API: 연결 실패\n"

        # LangSmith 상태
        status += f"✅ LangSmith: 활성화됨 (프로젝트: {self.langsmith_project})\n"

        # Tavily Search 상태
        status += f"{'✅' if self.search_available else '❌'} Tavily Search: {'활성화됨' if self.search_available else '비활성화됨'}\n"

        # 모델 상태
        status += f"✅ 이미지 분류: 활성화됨\n"
        status += f"{'✅' if self.blip_available else '❌'} BLIP 캡션: {'활성화됨' if self.blip_available else '비활성화됨'}\n"
        status += f"{'✅' if self.clip_available else '❌'} CLIP: {'활성화됨' if self.clip_available else '비활성화됨'}\n"

        return status

def create_gradio_interface():
    """GPT 연동 Gradio 웹 인터페이스 생성"""

    # AI 시스템 초기화
    ai_system = GPTMultimodalAI()

    # CSS 스타일
    css = """
    .gradio-container {
        max-width: 1400px !important;
    }
    .image-container {
        max-height: 400px;
    }
    .output-text {
        font-family: 'Malgun Gothic', Arial, sans-serif;
        line-height: 1.6;
    }
    .api-status {
        background-color: #f0f0f0;
        padding: 10px;
        border-radius: 5px;
        margin: 10px 0;
    }
    """

    with gr.Blocks(css=css, title="🤖 GPT & LangChain & Tavily 연동 AI", theme=gr.themes.Soft()) as demo:
        gr.Markdown(f"""
        # 🤖 GPT & LangChain & Tavily 연동 멀티모달 AI 시스템
        ### OpenAI GPT, LangChain, LangSmith, Tavily Search가 연동된 최고급 이미지 분석 시스템

        🚀 **주요 기능:**
        - 📊 **이미지 분류**: ResNet 기반 정확한 객체 분류
        - 🧠 **GPT 향상 설명**: BLIP + GPT API로 상세하고 창의적인 설명 생성
        - 💬 **지능형 질문답변**: 이미지 + 웹 검색 결합 AI 대화
        - 🔍 **Tavily 웹 검색**: 실시간 정보 검색 및 통합
        - 📈 **LangSmith 추적**: 대화 품질 모니터링 (프로젝트: {ai_system.langsmith_project})

        ⚠️ **보안 공지**: API 키가 코드에 직접 설정되어 있습니다. 운영 환경에서는 환경변수 사용을 권장합니다.
        """)

        # API 상태 표시
        with gr.Row():
            with gr.Column(scale=1):
                api_status = gr.Markdown(
                    value=ai_system.get_api_status(),
                    elem_classes=["api-status"]
                )
                refresh_status_btn = gr.Button("🔄 상태 새로고침", size="sm")

        with gr.Row():
            with gr.Column(scale=1):
                # 이미지 업로드
                image_input = gr.Image(
                    label="📷 이미지 업로드",
                    type="pil",
                    height=400
                )

                # 질문 입력
                question_input = gr.Textbox(
                    label="❓ 질문 입력 (GPT + Tavily 검색 결합 답변)",
                    placeholder="예: 이 이미지의 최신 정보를 찾아서 알려주세요",
                    lines=3
                )

                # 검색 테스트
                with gr.Accordion("🔍 Tavily 검색 테스트", open=False):
                    search_query_input = gr.Textbox(
                        label="검색어 입력",
                        placeholder="검색하고 싶은 내용을 입력하세요"
                    )
                    search_btn = gr.Button("🔍 웹 검색 실행")
                    search_output = gr.Textbox(
                        label="검색 결과",
                        lines=5
                    )

            with gr.Column(scale=2):
                # 탭 구성
                with gr.Tabs():
                    with gr.Tab("📊 이미지 분류"):
                        classify_btn = gr.Button("🔍 분류 시작", variant="primary", size="lg")
                        classification_output = gr.Markdown(
                            label="분류 결과",
                            elem_classes=["output-text"]
                        )

                    with gr.Tab("🧠 GPT 향상 설명"):
                        describe_btn = gr.Button("✨ GPT 설명 생성", variant="primary", size="lg")
                        description_output = gr.Markdown(
                            label="GPT 향상 설명",
                            elem_classes=["output-text"]
                        )

                    with gr.Tab("💬 GPT + Tavily 질문답변"):
                        answer_btn = gr.Button("🤖 GPT + 검색 답변", variant="primary", size="lg")
                        qa_output = gr.Markdown(
                            label="GPT + Tavily 답변",
                            elem_classes=["output-text"]
                        )

                    with gr.Tab("📈 대화 히스토리"):
                        clear_memory_btn = gr.Button("🗑️ 대화 기록 초기화", variant="secondary")
                        memory_output = gr.Markdown(
                            label="대화 기록",
                            elem_classes=["output-text"]
                        )

        # 샘플 이미지들
        gr.Markdown("### 🖼️ 샘플 이미지로 테스트해보세요!")

        sample_images = [
            "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/American_Eskimo_Dog.jpg/440px-American_Eskimo_Dog.jpg",
            "https://upload.wikimedia.org/wikipedia/commons/thumb/d/d9/Collage_of_Nine_Dogs.jpg/440px-Collage_of_Nine_Dogs.jpg",
            "https://upload.wikimedia.org/wikipedia/commons/thumb/6/68/Orange_tabby_cat_sitting_on_fallen_leaves-Hisashi-01A.jpg/440px-Orange_tabby_cat_sitting_on_fallen_leaves-Hisashi-01A.jpg"
        ]

        with gr.Row():
            sample_btn1 = gr.Button("🐕 강아지 샘플", size="sm")
            sample_btn2 = gr.Button("🐕‍🦺 여러 강아지", size="sm")
            sample_btn3 = gr.Button("🐱 고양이 샘플", size="sm")

        # 함수들 정의
        def show_memory():
            if ai_system.memory.chat_memory.messages:
                history = "📋 **최근 대화 기록:**\n\n"
                for i, msg in enumerate(ai_system.memory.chat_memory.messages[-10:]):  # 최근 10개만
                    if hasattr(msg, 'content'):
                        role = "🧑 사용자" if "Human" in str(type(msg)) else "🤖 AI"
                        history += f"**{role}:** {msg.content[:200]}{'...' if len(msg.content) > 200 else ''}\n\n"
                return history
            else:
                return "아직 대화 기록이 없습니다."

        def clear_memory():
            ai_system.memory.clear()
            return "🗑️ 대화 기록이 초기화되었습니다."

        # 이벤트 바인딩
        classify_btn.click(
            fn=ai_system.classify_image,
            inputs=[image_input],
            outputs=[classification_output]
        )

        describe_btn.click(
            fn=ai_system.generate_description_with_gpt,
            inputs=[image_input],
            outputs=[description_output]
        )

        answer_btn.click(
            fn=ai_system.answer_question_with_gpt_and_search,
            inputs=[image_input, question_input],
            outputs=[qa_output]
        )

        refresh_status_btn.click(
            fn=ai_system.get_api_status,
            inputs=[],
            outputs=[api_status]
        )

        search_btn.click(
            fn=ai_system.search_web,
            inputs=[search_query_input],
            outputs=[search_output]
        )

        clear_memory_btn.click(
            fn=clear_memory,
            inputs=[],
            outputs=[memory_output]
        )

        # 대화 기록 표시
        answer_btn.click(
            fn=show_memory,
            inputs=[],
            outputs=[memory_output]
        )

        # 샘플 이미지 바인딩
        sample_btn1.click(fn=lambda: sample_images[0], outputs=[image_input])
        sample_btn2.click(fn=lambda: sample_images[1], outputs=[image_input])
        sample_btn3.click(fn=lambda: sample_images[2], outputs=[image_input])

        # 사용법 안내
        gr.Markdown(f"""
        ---
        ### 📋 시스템 정보 및 사용법

        #### 🔧 **현재 설정**
        - **OpenAI API**: 직접 설정됨
        - **LangSmith 프로젝트**: {ai_system.langsmith_project}
        - **Tavily Search**: {'활성화됨' if ai_system.search_available else '비활성화됨'}
        - **추적 모드**: LangSmith 트레이싱 활성화

        #### 🎯 **고급 기능**
        1. **이미지 분류**: ResNet 기반 1000개 클래스 분류
        2. **GPT 향상 설명**: BLIP 기본 분석 + GPT의 창의적 해석
        3. **GPT + Tavily 질문답변**: 이미지 + 실시간 웹 검색 결합
        4. **Tavily 검색**: 독립적인 웹 검색 기능
        5. **대화 히스토리**: 연속적 컨텍스트 유지

        #### 💡 **고급 질문 예시**
        - "이 동물의 최신 연구 결과를 찾아서 알려주세요"
        - "이 장소의 현재 관광 정보를 검색해주세요"
        - "이 제품의 최신 리뷰나 뉴스를 찾아주세요"
        - "이 기술의 최근 발전 동향을 분석해주세요"

        #### 🔍 **LangSmith 모니터링**
        - 프로젝트: {ai_system.langsmith_project}
        - 모든 대화와 검색이 추적됩니다
        - 성능, 비용, 품질 메트릭 수집

        #### ⚡ **성능 최적화**
        - 이미지는 자동으로 최적 크기로 조정
        - 대화 기록은 효율성을 위해 최근 6개만 유지
        - 검색 결과는 관련성 높은 상위 3개만 사용

        ⚠️ **보안 참고**: 현재 API 키가 코드에 하드코딩되어 있습니다.
        운영 환경에서는 반드시 환경변수나 보안 키 관리 시스템을 사용하세요.
        """)

    return demo

# 실행 함수
def main():
    """메인 실행 함수"""
    print("\n" + "="*70)
    print("🚀 GPT & LangChain & Tavily 연동 멀티모달 AI 시스템")
    print("="*70)

    print(f"\n✅ API 키 설정 완료:")
    print(f"   • OpenAI API: {OPENAI_API_KEY[:20]}...")
    print(f"   • LangSmith 프로젝트: langgraph_5")
    print(f"   • Tavily API: {TAVILY_API_KEY[:20]}...")

    # 인터페이스 생성 및 실행
    demo = create_gradio_interface()

    print("\n🌐 웹 인터페이스를 시작합니다...")
    print("💡 브라우저에서 자동으로 열립니다!")
    print("🔗 수동 접속: http://localhost:17861")
    print("\n" + "="*70)

    # 공개 링크로 실행
    demo.launch(
        share=True,
        inbrowser=True,
        show_error=True,
        server_name="0.0.0.0",
        server_port=7860,
        height=800,
    )

if __name__ == "__main__":
    main()

✅ Tavily Search (langchain-teddynote) 로드 완료
✅ CLIP 라이브러리 로드 완료

🚀 GPT & LangChain & Tavily 연동 멀티모달 AI 시스템

✅ API 키 설정 완료:
   • OpenAI API: sk-proj-ZFU7TcfmbMzg...
   • LangSmith 프로젝트: langgraph_5
   • Tavily API: tvly-dev-r3bStTqv6qi...
🚀 GPT API 연동 AI 시스템 초기화 중...
🔑 API 서비스 설정 중...
✅ OpenAI API: 설정됨
✅ LangSmith: 설정됨 (프로젝트: langgraph_6)
✅ Tavily API: 설정됨
🖥️ 사용 디바이스: cpu
📦 이미지 분류 모델 로딩...
✅ ResNet 분류 모델 로드 완료
🔍 비전-언어 모델 로딩...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


✅ CLIP 모델 로드 완료


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

✅ BLIP 모델 로드 완료
🧠 GPT 및 LangChain 초기화...
✅ LangSmith 트레이싱 활성화 (프로젝트: langgraph_6)
✅ GPT 및 LangChain 초기화 완료
🔍 검색 도구 초기화...
✅ Tavily Search 도구 로드 완료
✅ AI 시스템 초기화 완료!

🌐 웹 인터페이스를 시작합니다...
💡 브라우저에서 자동으로 열립니다!
🔗 수동 접속: http://localhost:17861

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://46eb9af34a9987d580.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
